In [1]:
import torch
import numpy as np

In [96]:
class MCDC_FC(torch.nn.Module):
    def __init__(self, in_feat, out_feat, p):
        super().__init__()
        self.in_feat = in_feat
        self.out_feat= out_feat
        self.p = p
        W = torch.empty((in_feat, out_feat), requires_grad=True)
        bias = torch.empty(out_feat, requires_grad=True)

        torch.nn.init.xavier_normal_(W)
        torch.nn.init.normal_(bias)

        self.W = torch.nn.Parameter(W)
        self.bias = torch.nn.Parameter(bias)


    def forward(self, x):
        # mask_W = torch.ones_like(self.W, requires_grad=False)
        # print(mask_W)
        # mask_W = torch.nn.functional.dropout(mask_W, self.p, training=True) * self.p
        # print(mask_W)
        # mask_b = torch.ones_like(self.bias, requires_grad=False)
        # mask_b = torch.nn.functional.dropout(mask_b, self.p, training=True) * self.p

        print(self.W)
        print(self.bias)
        # print(self.W * mask_W)
        # print(self.bias * mask_b)
        W = torch.nn.functional.dropout(self.W, self.p, training=True) * self.p
        bias = torch.nn.functional.dropout(self.bias, self.p, training=True) * self.p
        print(W)
        print(bias)
        # x = torch.matmul(x, (self.W * mask_W))
        x = torch.matmul(x, (W))
        # x = x + self.bias*mask_b
        x = x + bias

        return x

In [97]:
layer = MCDC_FC(3, 2, 0.5)

In [98]:
fake_batch = torch.Tensor(np.random.randn(5, 3))
fake_gt = torch.Tensor(np.random.randn(5, 2))

In [99]:
optim = torch.optim.Adam(layer.parameters(), lr=0.01)

In [100]:
res = layer(fake_batch)

Parameter containing:
tensor([[ 0.8149,  0.6133],
        [-0.6334, -0.4846],
        [-1.2116,  0.1024]], requires_grad=True)
Parameter containing:
tensor([-0.4647, -0.9985], requires_grad=True)
tensor([[ 0.0000,  0.6133],
        [-0.0000, -0.4846],
        [-0.0000,  0.0000]], grad_fn=<MulBackward0>)
tensor([-0.4647, -0.0000], grad_fn=<MulBackward0>)


In [101]:
loss = torch.nn.functional.mse_loss(fake_gt, res)

In [102]:
loss.backward()
optim.step()

In [103]:
print(layer.W)
print(layer.bias)

Parameter containing:
tensor([[ 0.8149,  0.6033],
        [-0.6334, -0.4746],
        [-1.2116,  0.1024]], requires_grad=True)
Parameter containing:
tensor([-0.4547, -0.9985], requires_grad=True)
